In [29]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import datetime
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import matplotlib
#matplotlib.rcParams.update({'font.size': 16})
import matplotlib.ticker as mtick

In [2]:
import glob
file_list = glob.glob("data/*.tsv")
bad_files = ['data/SRR3722077_wgmlst_sig.tsv', 'data/SRR3722081_wgmlst_sig.tsv',
             'data/SRR3722145_wgmlst_sig.tsv']
file_list = list(filter(lambda x: x not in bad_files, file_list))
assert(file_list != [])

In [3]:
sample_info = pd.read_csv("sample_info.txt", sep='\t', names=["SRX", "SRR", "SAMPLE"])

srr_to_man_sample = {sample_info["SRR"][i]: sample_info["SAMPLE"][i]
                 for i in range(len(sample_info)) if "Manifold" in
                 sample_info["SAMPLE"][i] }

good_numbers = [163, 164, 166, 167, 189, 190, 196, 201, 202, 203, 204, 191, 192, 212, 213, 214, 215, 216, 193, 187, 188, 171, 179, 178]
digit = re.compile("[0-9]*$")
srr_to_man_sample_zeros = {list(srr_to_man_sample.keys())[i]: list(srr_to_man_sample.values())[i]
                           for i in range(len(srr_to_man_sample)) if int(digit.search(list(srr_to_man_sample.values())[i]).group(0)) in good_numbers}

srr_to_sym_sample = {sample_info["SRR"][i]: sample_info["SAMPLE"][i]
                 for i in range(len(sample_info)) if "Manifold" not in
                     sample_info["SAMPLE"][i]}
del(srr_to_sym_sample['SRR3722117'])

In [4]:
samples = dict(list(srr_to_sym_sample.items()) + list(srr_to_man_sample_zeros.items()))
labels = np.hstack((np.zeros(24), np.array([300, 300, 3000, 3000, 3000, 3000, 3000, 3000,
                   3000, 3000, 3000, 3000, 300, 3000, 3000, 3000, 3000, 3000, 3000, 3000])))

In [5]:
names = list(map(lambda sample: "data/" + sample + "_wgmlst_sig.tsv", samples.keys()))

In [6]:
sample_filenames = list(filter(lambda fn: fn in names, file_list))

In [18]:
bpmap = {"A": 0, "T": 1, "C": 2, "G": 3}
unbpmap = ["A", "T", "C", "G"]
def map_genome(base_pair):
    return bpmap[base_pair]

def unmap_genome(bp_num):
    return unbpmap[bp_num]

# Go from file name to list of length 3994, each entry w/ 330 base pair sequence
def file_to_matrix(file_name):
    genome_data = pd.read_csv(file_name, sep='\t',
                              names = ['foo', 'bar', 'genome'], skiprows=3)
    genome = genome_data['genome']
    # Flatten
    genome = genome.values.flatten()
    genome_len = len(genome)
    genome_vec = list(map(map_genome, list("".join(genome))))
    print(file_name)
    return genome_vec

In [8]:
data_matrices = list(map(file_to_matrix, sample_filenames))
# 241 samples, each with 1,318,020 base pairs


data/SRR3722097_wgmlst_sig.tsv


data/SRR3722138_wgmlst_sig.tsv


data/SRR3722132_wgmlst_sig.tsv


data/SRR3722135_wgmlst_sig.tsv


data/SRR3722121_wgmlst_sig.tsv


...

data/SRR3722071_wgmlst_sig.tsv


data/SRR3722131_wgmlst_sig.tsv


data/SRR3722052_wgmlst_sig.tsv


data/SRR3722096_wgmlst_sig.tsv


data/SRR3722129_wgmlst_sig.tsv


In [30]:


for i in range(1):
    chopped = list(map(lambda m: m[int(3e6)*i:int(3e6)*(i+1)], data_matrices))
    chopped = np.array(chopped)
    xTr, xVa = chopped[:38], chopped[38:]
    yTr, yVa = labels[:38], labels[38:]


    rf = RandomForestClassifier()
    rf.fit(xTr, yTr)
    print(i, rf.score(xVa, yVa))
    print(rf.predict(xVa))

    fig, ax = plt.subplots(figsize=(6,6))
    plt.plot(rf.feature_importances_*100)
    plt.xlabel('Location in Genome')
    plt.ylabel('Importance of Mutation for Resistance')

    fmt = '%.0f%%' # Format you want the ticks, e.g. '40%'
    yticks = mtick.FormatStrFormatter(fmt)
    ax.yaxis.set_major_formatter(yticks)

    plt.tight_layout()
    plt.savefig('ft_importances/' + str(datetime.datetime.now()) + "_" + str(i) + ".png")
    plt.close()

0 0.0
[0. 0. 0. 0. 0. 0.]


In [15]:
genome_data = pd.read_csv(file_list[0], sep='\t',
                          names = ['foo', 'bar', 'genome'], skiprows=3)
genome = genome_data['genome']
# Flatten
genome = genome.values.flatten()
genome_len = len(genome)
genome_vec = list(map(map_genome, list(genome[0])))
assert(file_list != [])